In [1]:
import os
import sys
import pickle
import psutil
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split

# Define data path
DATA_PATH = "data/"
CHECKPOINT_PATH = "models/"

In [2]:
pids = pickle.load(open(os.path.join(DATA_PATH,'pids.pkl'), 'rb'))
vids = pickle.load(open(os.path.join(DATA_PATH,'vids.pkl'), 'rb'))
targs = pickle.load(open(os.path.join(DATA_PATH,'targets.pkl'), 'rb'))
prob_targs = pickle.load(open(os.path.join(DATA_PATH, 'prob_targets_allvisits.pkl'), 'rb'))
seqs = pickle.load(open(os.path.join(DATA_PATH,'seqs.pkl'), 'rb'))
diags = pickle.load(open(os.path.join(DATA_PATH,'diags.pkl'), 'rb'))
codes = pickle.load(open(os.path.join(DATA_PATH,'icd9.pkl'), 'rb'))
categories = pickle.load(open(os.path.join(DATA_PATH,'categories.pkl'), 'rb'))
sub_categories = pickle.load(open(os.path.join(DATA_PATH,'subcategories.pkl'), 'rb'))
assert len(pids) == len(vids) == len(targs) == len(seqs)

In [3]:
embedding_matrix = torch.load(os.path.join(DATA_PATH, 'embedding_matrix.pt'))

In [4]:
class CustomDataset(Dataset):
    
    def __init__(self, seqs, targets):
        self.x = seqs
        self.y = targets
    
    def __len__(self): 
        return(len(self.x))
    
    def __getitem__(self, index):
        return (self.x[index], self.y[index])

In [5]:
# dataset = CustomDataset(seqs, targs)
dataset = CustomDataset(seqs, prob_targs)

In [6]:
def collate_fn(data):
    """
    Arguments:
        data: a list of samples fetched from `CustomDataset`
        
    Outputs:
        x: a tensor of shape (# patiens, max # visits, max # diagnosis codes) of type torch.long
        x_masks: a tensor of shape (# patiens, max # visits, max # diagnosis codes) of type torch.bool
        rev_x: same as x but in reversed time.
        rev_masks: same as x_masks but in reversed time.
        y: a tensor of shape (# patients, max # diagnosis categories) of type torch.float
        y_masks: a tensor of shape (# patients, max # diagnosis categories) of type torch.bool
    """
#     sequences, targets = zip(*data)

#     num_patients = len(sequences)
#     num_visits = [len(patient) for patient in sequences]
#     num_codes = [len(visit) for patient in sequences for visit in patient]
#     batch_num_categories = [len(visit) for patient in targets for visit in patient]

#     max_num_visits = max(num_visits)
#     max_num_codes = max(num_codes)
#     max_num_categories = max(batch_num_categories)
    
#     x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.long)
#     y = torch.zeros((num_patients, max_num_categories), dtype=torch.long)
#     rev_x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.long)
#     x_masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
#     rev_x_masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
#     y_masks = torch.zeros((num_patients, max_num_categories), dtype=torch.bool)
#     for i_patient, patient in enumerate(sequences):   
#         for j_visit, visit in enumerate(patient[:-1]):
#             for k_code, code in enumerate(visit):
#                 x[i_patient, j_visit, k_code] = code
#                 x_masks[i_patient, j_visit, k_code] = 1
#                 if j_visit == len(patient) - 2 and k_code == len(visit) - 1:
#                     rev_visit = x_masks[i_patient].any(dim=1)
#                     rev_x[i_patient, rev_visit] = x[i_patient, rev_visit].flip(0)
#                     rev_x_masks[i_patient, rev_visit] = x_masks[i_patient, rev_visit].flip(0)
                
#     for i_patient, patient in enumerate(targets):   
#         for visit in patient[-1:]:
#             for k_code, code in enumerate(visit):
#                 y[i_patient, k_code] = code
#                 y_masks[i_patient, k_code] = 1
    
#     return x, x_masks, rev_x, rev_x_masks, y, y_masks

    sequences, targets = zip(*data)

    num_patients = len(sequences)
    num_visits = [len(patient) for patient in sequences]
    num_codes = [len(visit) for patient in sequences for visit in patient]
    batch_num_categories = [len(visit) for patient in targets for visit in patient]
    max_num_visits = max(num_visits)
    max_num_codes = max(num_codes)
#     max_num_categories = max(batch_num_categories)
    max_num_categories = len(targets[0][0])
    
    x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.long)
    rev_x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.long)
    x_masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
    rev_x_masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
#     y = torch.zeros((num_patients, max_num_categories), dtype=torch.long)
    y = torch.zeros((num_patients, max_num_categories), dtype=torch.float)
    y_masks = torch.zeros((num_patients, max_num_categories), dtype=torch.bool)
    for i_patient, patient in enumerate(sequences):   
        for j_visit, visit in enumerate(patient[:-1]):
            for k_code, code in enumerate(visit):
                x[i_patient, j_visit, k_code] = code
                x_masks[i_patient, j_visit, k_code] = 1
                if j_visit == len(patient) - 2 and k_code == len(visit) - 1:
                    rev_visit = x_masks[i_patient].any(dim=1)
                    rev_x[i_patient, rev_visit] = x[i_patient, rev_visit].flip(0)
                    rev_x_masks[i_patient, rev_visit] = x_masks[i_patient, rev_visit].flip(0)

#     for i_patient, patient in enumerate(targets):
# #         import pdb; pdb.set_trace()
#         last_visit = patient[-1]
#         y[i_patient,:len(last_visit)] = torch.LongTensor(last_visit)
#         y_masks[i_patient,:len(last_visit)] = torch.BoolTensor(np.ones(len(last_visit)))
    for i_patient, patient in enumerate(targets):
#         import pdb; pdb.set_trace()
        last_visit = patient[-1]
        y[i_patient] = torch.FloatTensor(last_visit)
        y_masks[i_patient] = torch.BoolTensor(np.ones(max_num_categories))
    
    return x, x_masks, rev_x, rev_x_masks, y, y_masks

In [7]:
train_split = int(len(dataset)*0.75)
test_split = int(len(dataset)*0.15)
val_split = int(len(dataset)*0.10)

In [8]:
train_split = int(len(dataset)*0.75)
test_split = int(len(dataset)*0.15)

lengths = [train_split, test_split, len(dataset) - (train_split + test_split)]
train_dataset, test_dataset, val_dataset = random_split(dataset, lengths)

print("Length of train dataset:", len(train_dataset))
print("Length of test dataset:", len(test_dataset))
print("Length of val dataset:", len(val_dataset))

Length of train dataset: 6561
Length of test dataset: 1312
Length of val dataset: 875


In [9]:
def load_data(train_dataset, test_dataset, val_dataset, collate_fn):
    '''
    Arguments:
        train dataset: train dataset of type `CustomDataset`
        test dataset: test dataset of type `CustomDataset`
        val dataset: validation dataset of type `CustomDataset`
        collate_fn: collate function
        
    Outputs:
        train_loader, test_loader, val_loader: train, test and validation dataloaders
    '''
    batch_size = 100
    train_loader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=batch_size,
                                               collate_fn=collate_fn,
                                               shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset,
                                           batch_size=batch_size,
                                           collate_fn=collate_fn,
                                           shuffle=False)
    val_loader = torch.utils.data.DataLoader(val_dataset,
                                             batch_size=batch_size,
                                             collate_fn=collate_fn,
                                             shuffle=False)
    
    return train_loader, test_loader, val_loader


train_loader, test_loader, val_loader = load_data(train_dataset, test_dataset, val_dataset, collate_fn)

In [10]:
def sum_embeddings_with_mask(x, masks):
    """
    Arguments:
        x: the embeddings of diagnosis sequence of shape
            (batch_size, # visits, # diagnosis codes, embedding_dim)
        masks: the padding masks of shape (batch_size, # visits, # diagnosis codes)

    Outputs:
        sum_embeddings: the sum of embeddings of shape (batch_size, # visits, embedding_dim)
    """
    x[~masks] = 0
    return x.sum(2)

In [11]:
def attention_sum(alpha, v, masks):
    """
    Arguments:
        alpha: the alpha attention weights of shape (batch_size, # visits, 1)
        v: the concatentaion of forward and reversed visit embeddings
            of shape (batch_size, # visits, 2 * embedding_dim)
        masks: the concatentaion of forward and reversed padding masks
            of shape (batch_size, # visits, # diagnosis codes)

    Outputs:
        c: the context vector of shape (batch_size, 2 * embedding_dim)
    """
    masks = masks[:,:].any(dim=2)
    v_masked = v.clone()
    v_masked[~masks] = 0
    a_sum = torch.sum(alpha * v_masked, dim=1)
    return a_sum

In [12]:
class AlphaAttention(torch.nn.Module):

    def __init__(self, hidden_dim):
        super().__init__()
        """ 
        Arguments:
            hidden_dim: the hidden dimension
        """
        
        self.a_att = nn.Linear(hidden_dim, 1)

    def forward(self, g):
        """
        Arguments:
            g: the concatenated output tensors both forward and backward from RNN-alpha 
               of shape (batch_size, # visits, 2 * hidden_dim) 
        
        Outputs:
            alpha: the corresponding attention weights of shape (batch_size, # visits, 1)
        """
        m = nn.Softmax(dim=1)
        return m(self.a_att(g))

In [13]:
class EnhancedDipole(nn.Module):
    
    def __init__(self, num_codes, num_categories):
        super().__init__()
        """
        Arguments:
            num_codes: total number of diagnosis codes
            num_categories: total number of diagnosis categories to predict
        """
        self.embedding = nn.Linear(num_codes, 300)
        self.embedding.weight.data = embedding_matrix
        self.rnn = nn.GRU(300, hidden_size=128, batch_first=True)
        self.rev_rnn = nn.GRU(300, hidden_size=128, batch_first=True)
        self.att_a = AlphaAttention(256)
        self.fc = nn.Linear(600, num_categories)
    
    def forward(self, x, masks, rev_x, rev_masks):
        """
        Arguments:
            x: the diagnosis sequence of shape (batch_size, # visits, # diagnosis codes)
            masks: the padding masks of shape (batch_size, # visits, # diagnosis codes)
            rev_x: the diagnosis sequence of shape (batch_size, # visits, # diagnosis codes)
            rev_masks: the padding masks of shape (batch_size, # visits, # diagnosis codes)

        Outputs:
            logits: logits of shape (batch_size, # categories)
        """
        x = indices_to_multihot(x, masks, 4903)
        x = self.embedding(x)
        x[~masks.any(dim=2)] = 0  # Set masked visits to 0
        output, _ = self.rnn(x)
        
        rev_x = indices_to_multihot(rev_x, rev_masks, 4903)
        rev_x = self.embedding(rev_x)
        rev_x[~rev_masks.any(dim=2)] = 0  # Set masked visits to 0
        rev_output, _ = self.rev_rnn(rev_x)
        output_cat = torch.cat([output, rev_output], -1)
        x_cat = torch.cat([x, rev_x], -1)
        masks_cat = torch.cat([masks, rev_masks], -1)
        alpha = self.att_a(output_cat)
        c = attention_sum(alpha, x_cat, masks_cat)
        logits = self.fc(c)
        return logits
    

# load the model here
enhanced_dipole = EnhancedDipole(num_codes = len(codes), num_categories=len(sub_categories))
enhanced_dipole

EnhancedDipole(
  (embedding): Linear(in_features=4903, out_features=300, bias=True)
  (rnn): GRU(300, 128, batch_first=True)
  (rev_rnn): GRU(300, 128, batch_first=True)
  (att_a): AlphaAttention(
    (a_att): Linear(in_features=256, out_features=1, bias=True)
  )
  (fc): Linear(in_features=600, out_features=184, bias=True)
)

In [14]:
criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(naive_rnn.parameters(), lr=0.001)
optimizer = torch.optim.Adadelta(enhanced_dipole.parameters(), weight_decay=0.001)

In [15]:
def train(model, train_loader, test_loader, n_epochs):
    """
    Arguments:
        model: the EnhancedDipole model
        train_loader: training dataloder
        test_loader: validation dataloader
        n_epochs: total number of epochs
    """
#     max_cpu, max_ram = print_cpu_usage()
#     for epoch in range(n_epochs):
#         model.train()
#         train_loss = 0
#         for x, x_masks, rev_x, rev_x_masks, y, y_masks in train_loader:
#             y_hat = model(x, x_masks, rev_x, rev_x_masks) 
#             y_mh = indices_to_multihot(y, y_masks, y_hat.shape[-1])
#             loss = criterion(y_hat, y_mh)
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()
#             train_loss += loss.item()
#         train_loss = train_loss / len(train_loader)
#         cpu, ram = print_cpu_usage()
#         max_cpu = cpu if cpu > max_cpu else max_cpu
#         max_ram = ram if ram > max_ram else max_ram
#         print(f'Epoch: {epoch+1} \t Training Loss: {train_loss:.6f}')
#         for k in range(5, 31, 5):
#             precision_k, accuracy_k = eval_model(model, test_loader, k=k)
#             print(f'Epoch: {epoch+1} \t Validation precision@k{k}: {precision_k:.4f}, accuracy@k{k}: {accuracy_k:.4f}')
#     final_cpu, final_ram = print_cpu_usage()
#     print(f"Max CPU usage: {max_cpu:.3f}\tMax RAM % usage: {max_ram}")
    
    max_cpu, max_ram = print_cpu_usage()
    for epoch in range(n_epochs):
        model.train()
        train_loss = 0
        for x, x_masks, rev_x, rev_x_masks, y, y_masks in train_loader:
            y_hat = model(x, x_masks, rev_x, rev_x_masks)                  
#             y_mh = indices_to_multihot(y, y_masks, y_hat)
            loss = criterion(y_hat, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss = train_loss / len(train_loader)
        cpu, ram = print_cpu_usage()
        max_cpu = cpu if cpu > max_cpu else max_cpu
        max_ram = ram if ram > max_ram else max_ram
        print(f'Epoch: {epoch+1} \t Training Loss: {train_loss:.6f}')
        for k in range(5, 31, 5):
            precision_k, accuracy_k = eval_model(model, val_loader, k=k)
            print(f'Epoch: {epoch+1} \t Validation precision@k{k}: {precision_k:.4f}, accuracy@k{k}: {accuracy_k:.4f}')
    final_cpu, final_ram = print_cpu_usage()
    print(f"Max CPU usage: {max_cpu:.3f}\tMax RAM % usage: {max_ram}")

In [16]:
def eval_model(model, test_loader, k=15, n=-1):
    """
    Arguments:
        model: the EnhancedDipole model
        test_loader: validation dataloader
        k: value for top k predictions
        n: num of records to evaluate in the batch, value -1 evaulates all records
        
    Outputs:
        precision_k: visit-level precison@k
        accuracy_k: code-level accuracy@k
    """
#     y_pred = torch.LongTensor()
#     y_true = torch.LongTensor()
#     all_precision = []
#     all_accuracy = []
    
#     model.eval()
#     with torch.no_grad():
#         for x, x_masks, rev_x, rev_x_masks, y, y_masks in test_loader:
#             n_eval = y.shape[0] - 1 if n == -1 else n
#             y_hat = model(x, x_masks, rev_x, rev_x_masks)
#             y_hat = F.softmax(y_hat, dim=-1)
#             y_multihot = indices_to_multihot(y, y_masks, y_hat.shape[-1])
#             k_correct = 0
#             total_precision = 0
#             total_accuracy = 0
#             for i in range(n_eval):
#                 visit_correct = 0
#                 y_true = y[i, y_masks[i]]
#                 _, y_pred = torch.topk(y_hat[i], k)

#                 for v in y_true:
#                     if v in y_pred:
#                         visit_correct += 1
                        
#                 visit_precision = visit_correct / min(k, len(y_true))
#                 visit_accuracy = visit_correct / len(y_true)
#                 k_correct += visit_correct
#                 total_precision += visit_precision
#                 total_accuracy += visit_accuracy

#             precision_k = total_precision / n_eval
#             accuracy_k = total_accuracy / n_eval
#             all_precision.append(precision_k)
#             all_accuracy.append(accuracy_k)

#     total_precision_k = np.mean(all_precision)
#     total_accuracy_k = np.mean(all_accuracy)
#     return total_precision_k, total_accuracy_k

    y_pred = torch.LongTensor()
    y_true = torch.LongTensor()
    all_precision = []
    all_accuracy = []
    
    model.eval()
    with torch.no_grad():
        for x, x_masks, rev_x, rev_x_masks, y, y_masks in test_loader:
            n_eval = y.shape[0] - 1 if n == -1 else n
            y_hat = model(x, x_masks, rev_x, rev_x_masks)
            y_hat = F.softmax(y_hat, dim=-1)
            #y_multihot = indices_to_multihot(y, y_masks, y_hat)
            nz_rows, nz_cols = torch.nonzero(y, as_tuple=True)
            k_correct = 0
            total_precision = 0
            total_accuracy = 0
            for i in range(n_eval):
                visit_correct = 0
                #y_true = y[i, y_masks[i]]
                y_true = nz_cols[nz_rows == i]
                _, y_pred = torch.topk(y_hat[i], k)
                for v in y_true:
                    if v in y_pred:
                        visit_correct += 1
                visit_precision = visit_correct / min(k, len(y_true))
                visit_accuracy = visit_correct / len(y_true)
                k_correct += visit_correct
                total_precision += visit_precision
                total_accuracy += visit_accuracy
            precision_k = total_precision / n_eval
            accuracy_k = total_accuracy / n_eval
            all_precision.append(precision_k)
            all_accuracy.append(accuracy_k)
            
    total_precision_k = np.mean(all_precision)
    total_accuracy_k = np.mean(all_accuracy)
    return total_precision_k, total_accuracy_k

In [17]:
def indices_to_multihot(indices, masks, dim):
    mh_size = tuple(indices.shape[:-1]) + (dim,)
    multihot = torch.zeros(mh_size, dtype=torch.float)
    if len(mh_size) == 3:
        for i, patient in enumerate(indices):
            for j, visit in enumerate(patient):
                if masks[i,j].sum() == 0:
                    break
                y_idx = visit[masks[i,j]].unique()
                multihot[i,j] = F.one_hot(y_idx.to(torch.int64), multihot.shape[-1]).sum(0)
    else:
        for idx, row in enumerate(indices):
            y_idx = row[masks[idx]].unique()
            multihot[idx] = F.one_hot(y_idx, dim).sum(0).float()
    return multihot

In [18]:
def print_cpu_usage():
    load = psutil.getloadavg()[2]
    cpu_usage = (load/os.cpu_count()) * 100
    ram = psutil.virtual_memory()[2]
    print(f"CPU: {cpu_usage:0.2f}")
    print(f"RAM %: {ram}")
    return cpu_usage, ram

In [19]:
n_epochs = 100
%time train(enhanced_dipole, train_loader, val_loader, n_epochs)

CPU: 17.43
RAM %: 61.9
CPU: 17.50
RAM %: 62.1
Epoch: 1 	 Training Loss: 4.526846
Epoch: 1 	 Validation precision@k5: 0.4220, accuracy@k5: 0.2092
Epoch: 1 	 Validation precision@k10: 0.4236, accuracy@k10: 0.3545
Epoch: 1 	 Validation precision@k15: 0.4810, accuracy@k15: 0.4640
Epoch: 1 	 Validation precision@k20: 0.5662, accuracy@k20: 0.5636
Epoch: 1 	 Validation precision@k25: 0.6493, accuracy@k25: 0.6490
Epoch: 1 	 Validation precision@k30: 0.7052, accuracy@k30: 0.7052
CPU: 18.19
RAM %: 62.1
Epoch: 2 	 Training Loss: 4.271104
Epoch: 2 	 Validation precision@k5: 0.4875, accuracy@k5: 0.2460
Epoch: 2 	 Validation precision@k10: 0.4834, accuracy@k10: 0.4092
Epoch: 2 	 Validation precision@k15: 0.5512, accuracy@k15: 0.5338
Epoch: 2 	 Validation precision@k20: 0.6354, accuracy@k20: 0.6328
Epoch: 2 	 Validation precision@k25: 0.7047, accuracy@k25: 0.7043
Epoch: 2 	 Validation precision@k30: 0.7571, accuracy@k30: 0.7571
CPU: 18.21
RAM %: 62.1
Epoch: 3 	 Training Loss: 4.136872
Epoch: 3 	 Vali

CPU: 24.56
RAM %: 62.2
Epoch: 19 	 Training Loss: 3.596718
Epoch: 19 	 Validation precision@k5: 0.5857, accuracy@k5: 0.3066
Epoch: 19 	 Validation precision@k10: 0.5641, accuracy@k10: 0.4804
Epoch: 19 	 Validation precision@k15: 0.6060, accuracy@k15: 0.5870
Epoch: 19 	 Validation precision@k20: 0.6772, accuracy@k20: 0.6745
Epoch: 19 	 Validation precision@k25: 0.7446, accuracy@k25: 0.7443
Epoch: 19 	 Validation precision@k30: 0.7942, accuracy@k30: 0.7942
CPU: 24.99
RAM %: 62.2
Epoch: 20 	 Training Loss: 3.593848
Epoch: 20 	 Validation precision@k5: 0.5854, accuracy@k5: 0.3081
Epoch: 20 	 Validation precision@k10: 0.5778, accuracy@k10: 0.4932
Epoch: 20 	 Validation precision@k15: 0.6409, accuracy@k15: 0.6212
Epoch: 20 	 Validation precision@k20: 0.7035, accuracy@k20: 0.7006
Epoch: 20 	 Validation precision@k25: 0.7629, accuracy@k25: 0.7625
Epoch: 20 	 Validation precision@k30: 0.8076, accuracy@k30: 0.8076
CPU: 24.90
RAM %: 62.3
Epoch: 21 	 Training Loss: 3.578175
Epoch: 21 	 Validation 

Epoch: 36 	 Validation precision@k30: 0.8308, accuracy@k30: 0.8308
CPU: 31.48
RAM %: 62.4
Epoch: 37 	 Training Loss: 3.380568
Epoch: 37 	 Validation precision@k5: 0.6771, accuracy@k5: 0.3572
Epoch: 37 	 Validation precision@k10: 0.6437, accuracy@k10: 0.5505
Epoch: 37 	 Validation precision@k15: 0.6840, accuracy@k15: 0.6633
Epoch: 37 	 Validation precision@k20: 0.7445, accuracy@k20: 0.7416
Epoch: 37 	 Validation precision@k25: 0.7967, accuracy@k25: 0.7963
Epoch: 37 	 Validation precision@k30: 0.8368, accuracy@k30: 0.8368
CPU: 30.75
RAM %: 62.4
Epoch: 38 	 Training Loss: 3.367366
Epoch: 38 	 Validation precision@k5: 0.6772, accuracy@k5: 0.3570
Epoch: 38 	 Validation precision@k10: 0.6402, accuracy@k10: 0.5457
Epoch: 38 	 Validation precision@k15: 0.6889, accuracy@k15: 0.6673
Epoch: 38 	 Validation precision@k20: 0.7438, accuracy@k20: 0.7409
Epoch: 38 	 Validation precision@k25: 0.7951, accuracy@k25: 0.7947
Epoch: 38 	 Validation precision@k30: 0.8352, accuracy@k30: 0.8352
CPU: 30.40
RAM 

Epoch: 54 	 Validation precision@k25: 0.8091, accuracy@k25: 0.8087
Epoch: 54 	 Validation precision@k30: 0.8467, accuracy@k30: 0.8467
CPU: 25.66
RAM %: 62.3
Epoch: 55 	 Training Loss: 3.281486
Epoch: 55 	 Validation precision@k5: 0.7072, accuracy@k5: 0.3693
Epoch: 55 	 Validation precision@k10: 0.6715, accuracy@k10: 0.5703
Epoch: 55 	 Validation precision@k15: 0.7003, accuracy@k15: 0.6775
Epoch: 55 	 Validation precision@k20: 0.7526, accuracy@k20: 0.7494
Epoch: 55 	 Validation precision@k25: 0.8074, accuracy@k25: 0.8070
Epoch: 55 	 Validation precision@k30: 0.8457, accuracy@k30: 0.8457
CPU: 25.52
RAM %: 62.2
Epoch: 56 	 Training Loss: 3.277661
Epoch: 56 	 Validation precision@k5: 0.7016, accuracy@k5: 0.3670
Epoch: 56 	 Validation precision@k10: 0.6736, accuracy@k10: 0.5731
Epoch: 56 	 Validation precision@k15: 0.7022, accuracy@k15: 0.6792
Epoch: 56 	 Validation precision@k20: 0.7524, accuracy@k20: 0.7492
Epoch: 56 	 Validation precision@k25: 0.8059, accuracy@k25: 0.8055
Epoch: 56 	 Val

Epoch: 72 	 Validation precision@k20: 0.7580, accuracy@k20: 0.7546
Epoch: 72 	 Validation precision@k25: 0.8083, accuracy@k25: 0.8079
Epoch: 72 	 Validation precision@k30: 0.8493, accuracy@k30: 0.8493
CPU: 21.68
RAM %: 62.1
Epoch: 73 	 Training Loss: 3.262593
Epoch: 73 	 Validation precision@k5: 0.6824, accuracy@k5: 0.3563
Epoch: 73 	 Validation precision@k10: 0.6719, accuracy@k10: 0.5708
Epoch: 73 	 Validation precision@k15: 0.7019, accuracy@k15: 0.6788
Epoch: 73 	 Validation precision@k20: 0.7590, accuracy@k20: 0.7556
Epoch: 73 	 Validation precision@k25: 0.8086, accuracy@k25: 0.8082
Epoch: 73 	 Validation precision@k30: 0.8483, accuracy@k30: 0.8483
CPU: 21.41
RAM %: 62.3
Epoch: 74 	 Training Loss: 3.262699
Epoch: 74 	 Validation precision@k5: 0.6939, accuracy@k5: 0.3617
Epoch: 74 	 Validation precision@k10: 0.6718, accuracy@k10: 0.5711
Epoch: 74 	 Validation precision@k15: 0.7056, accuracy@k15: 0.6825
Epoch: 74 	 Validation precision@k20: 0.7579, accuracy@k20: 0.7546
Epoch: 74 	 Val

Epoch: 90 	 Validation precision@k15: 0.7023, accuracy@k15: 0.6791
Epoch: 90 	 Validation precision@k20: 0.7551, accuracy@k20: 0.7518
Epoch: 90 	 Validation precision@k25: 0.8055, accuracy@k25: 0.8050
Epoch: 90 	 Validation precision@k30: 0.8473, accuracy@k30: 0.8473
CPU: 20.91
RAM %: 62.2
Epoch: 91 	 Training Loss: 3.257912
Epoch: 91 	 Validation precision@k5: 0.7014, accuracy@k5: 0.3674
Epoch: 91 	 Validation precision@k10: 0.6709, accuracy@k10: 0.5700
Epoch: 91 	 Validation precision@k15: 0.7031, accuracy@k15: 0.6800
Epoch: 91 	 Validation precision@k20: 0.7556, accuracy@k20: 0.7523
Epoch: 91 	 Validation precision@k25: 0.8076, accuracy@k25: 0.8072
Epoch: 91 	 Validation precision@k30: 0.8476, accuracy@k30: 0.8476
CPU: 20.80
RAM %: 62.1
Epoch: 92 	 Training Loss: 3.257508
Epoch: 92 	 Validation precision@k5: 0.6795, accuracy@k5: 0.3561
Epoch: 92 	 Validation precision@k10: 0.6633, accuracy@k10: 0.5641
Epoch: 92 	 Validation precision@k15: 0.7018, accuracy@k15: 0.6788
Epoch: 92 	 Val

In [20]:
for k in range(5, 31, 5):
    precision_k, accuracy_k = eval_model(enhanced_dipole, test_loader, k=k)
    print(f'Validation precision@k{k}: {precision_k:.4f}, accuracy@k{k}: {accuracy_k:.4f}')

Validation precision@k5: 0.7217, accuracy@k5: 0.3937
Validation precision@k10: 0.6820, accuracy@k10: 0.5812
Validation precision@k15: 0.7164, accuracy@k15: 0.6899
Validation precision@k20: 0.7709, accuracy@k20: 0.7651
Validation precision@k25: 0.8154, accuracy@k25: 0.8148
Validation precision@k30: 0.8503, accuracy@k30: 0.8503


In [21]:
torch.save(enhanced_dipole, os.path.join(CHECKPOINT_PATH, "EnhancedDipole_100.pth"))